In [ ]:
%load_ext autoreload

In [5]:
import sqlite3
import json
from datetime import datetime
import lzma

In [6]:
timeframe = '2018-09'
sql_transactions = []
start_row = 0
cleanup = 1000000
connect = sqlite3.connect("chatbot/db/RC_{}.db".format(timeframe))
c = connect.cursor()

In [ ]:
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS data_201809(parent_id TEXT PRIMARY KEY,comment_id TEXT UNIQUE, parent TEXT, comment TEXT, score INT, unix INT, subreddit TEXT)""")
create_table()

In [114]:
print(c.execute("SELECT * FROM data_201809").fetchall())

OperationalError: no such table: data_201809

In [ ]:
x = {'archived': False, 'author': 'kmoz', 'author_created_utc': 1302834614, 'author_flair_background_color': '', 'author_flair_css_class': '', 'author_flair_richtext': [], 'author_flair_template_id': None, 'author_flair_text': 'CPI', 'author_flair_text_color': 'dark', 'author_flair_type': 'text', 'author_fullname': 't2_53u56', 'body': 'Seconding this, Ive used it on some pretty large data sets without issue. ', 'can_gild': True, 'can_mod_post': False, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0, 'created_utc': 1535760000, 'distinguished': None, 'edited': False, 'gilded': 0, 'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0}, 'id': 'e56q3qi', 'is_submitter': False, 'link_id': 't3_9bs9tk', 'no_follow': True, 'parent_id': 't1_e55cz9g', 'permalink': '/r/LabVIEW/comments/9bs9tk/finding_local_maximaminima_in_a_1darray/e56q3qi/', 'removal_reason': None, 'retrieved_on': 1538538086, 'score': 1, 'send_replies': True, 'stickied': False, 'subreddit': 'LabVIEW', 'subreddit_id': 't5_2rk9b', 'subreddit_name_prefixed': 'r/LabVIEW', 'subreddit_type': 'public'}
x

In [113]:
import re
import unicodedata


def format_data(data):
    data = data.replace('\n',' ').replace('\r',' ').replace('"',"'")
    return preprocess_sentence(data)

def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False
    
row_counter = 0
paired_rows = 0

with lzma.open('RC/RC_{}.xz'.format(timeframe)) as f:
    for row in f:
        #print(row)
        row_counter += 1

        if row_counter > start_row:
            try:
                row = json.loads(row)
                parent_id = row['parent_id'].split('_')[1]
                body = format_data(row['body'])
                created_utc = row['created_utc']
                score = row['score']

                comment_id = row['id']

                subreddit = row['subreddit']
                parent_data = find_parent(parent_id)

                existing_comment_score = find_existing_score(parent_id)
                if existing_comment_score:
                    if score > existing_comment_score:
                        if acceptable(body):
                            sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)

                else:
                    if acceptable(body):
                        if parent_data:
                            if score >= 2:
                                sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
            except Exception as e:
                print(str(e))

        if row_counter % 100000 == 0:
            print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

        if row_counter > start_row:
            if row_counter % cleanup == 0:
                print("Cleanin up!")
                sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                c.execute(sql)
                connection.commit()
#                     c.execute("VACUUM")
#                     connection.commit()


Total Rows Read: 100000, Paired Rows: 6333, Time: 2019-05-26 03:06:41.763092
Total Rows Read: 200000, Paired Rows: 15227, Time: 2019-05-26 03:06:58.438649
Total Rows Read: 300000, Paired Rows: 24744, Time: 2019-05-26 03:07:14.686514
Total Rows Read: 400000, Paired Rows: 34428, Time: 2019-05-26 03:07:30.869413
Total Rows Read: 500000, Paired Rows: 44373, Time: 2019-05-26 03:07:47.459644
Total Rows Read: 600000, Paired Rows: 54457, Time: 2019-05-26 03:08:04.492717
Total Rows Read: 700000, Paired Rows: 64581, Time: 2019-05-26 03:08:20.730658
Total Rows Read: 800000, Paired Rows: 74406, Time: 2019-05-26 03:08:37.691019
Total Rows Read: 900000, Paired Rows: 84074, Time: 2019-05-26 03:08:54.705578
Total Rows Read: 1000000, Paired Rows: 93600, Time: 2019-05-26 03:09:11.468827
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 100951, Time: 2019-05-26 03:09:32.830694
Total Rows Read: 1200000, Paired Rows: 109558, Time: 2019-05-26 03:09:50.828293
Total Rows Read: 1300000, Paired Rows: 118633, T

Total Rows Read: 10300000, Paired Rows: 1037611, Time: 2019-05-26 03:36:09.598269
Total Rows Read: 10400000, Paired Rows: 1047563, Time: 2019-05-26 03:36:26.062649
Total Rows Read: 10500000, Paired Rows: 1057420, Time: 2019-05-26 03:36:42.657591
Total Rows Read: 10600000, Paired Rows: 1067220, Time: 2019-05-26 03:36:59.390821
Total Rows Read: 10700000, Paired Rows: 1077022, Time: 2019-05-26 03:37:16.175060
Total Rows Read: 10800000, Paired Rows: 1086958, Time: 2019-05-26 03:37:32.949745
Total Rows Read: 10900000, Paired Rows: 1097588, Time: 2019-05-26 03:37:49.448057
Total Rows Read: 11000000, Paired Rows: 1108314, Time: 2019-05-26 03:38:05.888247
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 1116876, Time: 2019-05-26 03:38:26.548119
Total Rows Read: 11200000, Paired Rows: 1126942, Time: 2019-05-26 03:38:43.829707
Total Rows Read: 11300000, Paired Rows: 1137442, Time: 2019-05-26 03:39:00.853436
Total Rows Read: 11400000, Paired Rows: 1148216, Time: 2019-05-26 03:39:17.874649
Tota

Total Rows Read: 20200000, Paired Rows: 2077100, Time: 2019-05-26 04:05:39.406222
Total Rows Read: 20300000, Paired Rows: 2087139, Time: 2019-05-26 04:05:56.683995
Total Rows Read: 20400000, Paired Rows: 2097135, Time: 2019-05-26 04:06:13.443045
Total Rows Read: 20500000, Paired Rows: 2106595, Time: 2019-05-26 04:06:29.719591
Total Rows Read: 20600000, Paired Rows: 2116222, Time: 2019-05-26 04:06:46.015469
Total Rows Read: 20700000, Paired Rows: 2125772, Time: 2019-05-26 04:07:01.694120
Total Rows Read: 20800000, Paired Rows: 2135106, Time: 2019-05-26 04:07:17.766760
Total Rows Read: 20900000, Paired Rows: 2144149, Time: 2019-05-26 04:07:33.820359
Total Rows Read: 21000000, Paired Rows: 2153489, Time: 2019-05-26 04:07:50.419331
Cleanin up!
Total Rows Read: 21100000, Paired Rows: 2161321, Time: 2019-05-26 04:08:12.598373
Total Rows Read: 21200000, Paired Rows: 2170412, Time: 2019-05-26 04:08:31.169202
Total Rows Read: 21300000, Paired Rows: 2179830, Time: 2019-05-26 04:08:48.806403
Tota

Total Rows Read: 30100000, Paired Rows: 3053612, Time: 2019-05-26 04:34:22.586216
Total Rows Read: 30200000, Paired Rows: 3062904, Time: 2019-05-26 04:34:39.366971
Total Rows Read: 30300000, Paired Rows: 3072876, Time: 2019-05-26 04:34:56.568771
Total Rows Read: 30400000, Paired Rows: 3082501, Time: 2019-05-26 04:35:12.765188
Total Rows Read: 30500000, Paired Rows: 3091423, Time: 2019-05-26 04:35:28.392955
Total Rows Read: 30600000, Paired Rows: 3101420, Time: 2019-05-26 04:35:44.320572
Total Rows Read: 30700000, Paired Rows: 3111284, Time: 2019-05-26 04:36:00.612514
Total Rows Read: 30800000, Paired Rows: 3120255, Time: 2019-05-26 04:36:16.350178
Total Rows Read: 30900000, Paired Rows: 3131151, Time: 2019-05-26 04:36:32.776514
Total Rows Read: 31000000, Paired Rows: 3142109, Time: 2019-05-26 04:36:49.519589
Cleanin up!
Total Rows Read: 31100000, Paired Rows: 3150416, Time: 2019-05-26 04:37:12.010414
Total Rows Read: 31200000, Paired Rows: 3159688, Time: 2019-05-26 04:37:29.360815
Tota

Total Rows Read: 40000000, Paired Rows: 4081830, Time: 2019-05-26 05:06:04.594920
Cleanin up!
Total Rows Read: 40100000, Paired Rows: 4090545, Time: 2019-05-26 05:06:35.742929
Total Rows Read: 40200000, Paired Rows: 4101193, Time: 2019-05-26 05:06:56.214676
Total Rows Read: 40300000, Paired Rows: 4112207, Time: 2019-05-26 05:07:16.225756
Total Rows Read: 40400000, Paired Rows: 4123492, Time: 2019-05-26 05:07:36.140154
Total Rows Read: 40500000, Paired Rows: 4134898, Time: 2019-05-26 05:07:58.357999
Total Rows Read: 40600000, Paired Rows: 4146679, Time: 2019-05-26 05:08:19.698974
Total Rows Read: 40700000, Paired Rows: 4158023, Time: 2019-05-26 05:08:39.724226
Total Rows Read: 40800000, Paired Rows: 4169006, Time: 2019-05-26 05:08:58.473294
Total Rows Read: 40900000, Paired Rows: 4180095, Time: 2019-05-26 05:09:18.761148
Total Rows Read: 41000000, Paired Rows: 4190987, Time: 2019-05-26 05:09:37.236825
Cleanin up!
Total Rows Read: 41100000, Paired Rows: 4199306, Time: 2019-05-26 05:10:01

Total Rows Read: 49900000, Paired Rows: 5099991, Time: 2019-05-26 05:41:12.825238
Total Rows Read: 50000000, Paired Rows: 5110549, Time: 2019-05-26 05:41:33.534797
Cleanin up!
Total Rows Read: 50100000, Paired Rows: 5118394, Time: 2019-05-26 05:41:59.153525
Total Rows Read: 50200000, Paired Rows: 5128226, Time: 2019-05-26 05:42:20.216601
Total Rows Read: 50300000, Paired Rows: 5138323, Time: 2019-05-26 05:42:40.337809
Total Rows Read: 50400000, Paired Rows: 5148336, Time: 2019-05-26 05:42:59.245776
Total Rows Read: 50500000, Paired Rows: 5158788, Time: 2019-05-26 05:43:17.399274
Total Rows Read: 50600000, Paired Rows: 5169423, Time: 2019-05-26 05:43:35.522626
Total Rows Read: 50700000, Paired Rows: 5180092, Time: 2019-05-26 05:43:55.191546
Total Rows Read: 50800000, Paired Rows: 5190604, Time: 2019-05-26 05:44:14.895398
Total Rows Read: 50900000, Paired Rows: 5200983, Time: 2019-05-26 05:44:34.758891
Total Rows Read: 51000000, Paired Rows: 5211598, Time: 2019-05-26 05:44:53.067566
Clea

KeyboardInterrupt: 

In [8]:
connect = sqlite3.connect("{}.db".format(timeframe))
c = connect.cursor()
c.execute("SELECT * FROM parent_reply LIMIT 5000")
result = c.fetchall()
result

[('e56q3v7',
  'e56q4st',
  'What phone do you have?',
  'Huawei nova lol',
  'DragonballLegends',
  1535760031,
  2),
 ('e56q4iq',
  'e56q5gq',
  'monday',
  'On monday you will hear of me.',
  'teenagersnew',
  1535760051,
  2),
 ('e56q56d',
  'e56q5hb',
  'I am a fellow. r/praisethecube and I agree to alliance against r/shrineofdurr I am a cube ambassador seeking peace with the tomato heads',
  "Here's a sneak peek of /r/PraiseTheCube using the [top posts](https://np.reddit.com/r/PraiseTheCube/top/?sort=top&amp;t=all) of all time! newlinechar  newlinechar \\#1: [Praise The C U B E](https://i.redd.it/rexmxyoyz3j11.jpg) | [8 comments](https://np.reddit.com/r/PraiseTheCube/comments/9bdz7s/praise_the_c_u_b_e/)   newlinechar \\#2: [praise the ć̇̌̑̏ͫ͋ͤ̆ͮͣ͂̈́̈́͐̆̐͑͏̴̞̝͖̞͢ų̴͉͖̩̙̝̘̖̥̤̘̂͑̌ͦ́͘͝ͅͅb̢̛ͥ̅ͨ̿̅ͩͩ̈́̒ͯ̀ͬ̿̒͂̿̈̀͘͏̥̰̣̬̲̩̺̹̱̮̙͖̬ë̸̇͑̓ͫͬ̌̂͏̵̡͕͙̭̤̦̳̲̠͔͟ͅ, for he has given me trickshot powers](https://gfycat.com/WanRashFlyingsquirrel) | [22 comments](https://np.reddit.com/r/PraiseTheCube/comme

In [4]:
type(result[0])

tuple

In [9]:
test = []
for x in result:
    test.append(x[2])
    test.append(x[3])
test

['What phone do you have?',
 'Huawei nova lol',
 'monday',
 'On monday you will hear of me.',
 'I am a fellow. r/praisethecube and I agree to alliance against r/shrineofdurr I am a cube ambassador seeking peace with the tomato heads',
 "Here's a sneak peek of /r/PraiseTheCube using the [top posts](https://np.reddit.com/r/PraiseTheCube/top/?sort=top&amp;t=all) of all time! newlinechar  newlinechar \\#1: [Praise The C U B E](https://i.redd.it/rexmxyoyz3j11.jpg) | [8 comments](https://np.reddit.com/r/PraiseTheCube/comments/9bdz7s/praise_the_c_u_b_e/)   newlinechar \\#2: [praise the ć̇̌̑̏ͫ͋ͤ̆ͮͣ͂̈́̈́͐̆̐͑͏̴̞̝͖̞͢ų̴͉͖̩̙̝̘̖̥̤̘̂͑̌ͦ́͘͝ͅͅb̢̛ͥ̅ͨ̿̅ͩͩ̈́̒ͯ̀ͬ̿̒͂̿̈̀͘͏̥̰̣̬̲̩̺̹̱̮̙͖̬ë̸̇͑̓ͫͬ̌̂͏̵̡͕͙̭̤̦̳̲̠͔͟ͅ, for he has given me trickshot powers](https://gfycat.com/WanRashFlyingsquirrel) | [22 comments](https://np.reddit.com/r/PraiseTheCube/comments/9b973i/praise_the_cube_for_he_has_given_me_trickshot/)   newlinechar \\#3: [The Cube is love, The Cube is life.](https://i.redd.it/vfep3b9zeri11.jpg) | [5 commen

In [61]:
unico = [_ for _ in test if "](https://" in _]
unico

["Here's a sneak peek of /r/PraiseTheCube using the [top posts](https://np.reddit.com/r/PraiseTheCube/top/?sort=top&amp;t=all) of all time! newlinechar  newlinechar \\#1: [Praise The C U B E](https://i.redd.it/rexmxyoyz3j11.jpg) | [8 comments](https://np.reddit.com/r/PraiseTheCube/comments/9bdz7s/praise_the_c_u_b_e/)   newlinechar \\#2: [praise the ć̇̌̑̏ͫ͋ͤ̆ͮͣ͂̈́̈́͐̆̐͑͏̴̞̝͖̞͢ų̴͉͖̩̙̝̘̖̥̤̘̂͑̌ͦ́͘͝ͅͅb̢̛ͥ̅ͨ̿̅ͩͩ̈́̒ͯ̀ͬ̿̒͂̿̈̀͘͏̥̰̣̬̲̩̺̹̱̮̙͖̬ë̸̇͑̓ͫͬ̌̂͏̵̡͕͙̭̤̦̳̲̠͔͟ͅ, for he has given me trickshot powers](https://gfycat.com/WanRashFlyingsquirrel) | [22 comments](https://np.reddit.com/r/PraiseTheCube/comments/9b973i/praise_the_cube_for_he_has_given_me_trickshot/)   newlinechar \\#3: [The Cube is love, The Cube is life.](https://i.redd.it/vfep3b9zeri11.jpg) | [5 comments](https://np.reddit.com/r/PraiseTheCube/comments/9avtgb/the_cube_is_love_the_cube_is_life/) newlinechar  newlinechar ---- newlinechar ^^I'm ^^a ^^bot, ^^beep ^^boop ^^| ^^Downvote ^^to ^^remove ^^| [^^Contact ^^me](https://www.reddit

In [82]:
[unicode_to_ascii(_) for _ in unico][0]

"Here's a sneak peek of /r/PraiseTheCube using the [top posts](https://np.reddit.com/r/PraiseTheCube/top/?sort=top&amp;t=all) of all time! newlinechar  newlinechar \\#1: [Praise The C U B E](https://i.redd.it/rexmxyoyz3j11.jpg) | [8 comments](https://np.reddit.com/r/PraiseTheCube/comments/9bdz7s/praise_the_c_u_b_e/)   newlinechar \\#2: [praise the cube, for he has given me trickshot powers](https://gfycat.com/WanRashFlyingsquirrel) | [22 comments](https://np.reddit.com/r/PraiseTheCube/comments/9b973i/praise_the_cube_for_he_has_given_me_trickshot/)   newlinechar \\#3: [The Cube is love, The Cube is life.](https://i.redd.it/vfep3b9zeri11.jpg) | [5 comments](https://np.reddit.com/r/PraiseTheCube/comments/9avtgb/the_cube_is_love_the_cube_is_life/) newlinechar  newlinechar ---- newlinechar ^^I'm ^^a ^^bot, ^^beep ^^boop ^^| ^^Downvote ^^to ^^remove ^^| [^^Contact ^^me](https://www.reddit.com/message/compose/?to=sneakpeekbot) ^^| [^^Info](https://np.reddit.com/r/sneakpeekbot/) ^^| [^^Opt-out

In [110]:
import re
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"(\[[a-zA-Z0-9 ,.\^-]*\]\(https:\/\/[\/?=&;a-zA-Z0-9._]*\))", " ", w)
    w = re.sub(r"(\^\^[\w*\|]*)"," ",w)
    w = re.sub(r"([\\\\\#[0-9]*\:]*)"," ", w)
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w

In [111]:
y = zip([preprocess_sentence(_) for _ in test], test)
y

In [112]:
[_[0] for _ in y]

['<start> what phone do you have ? <end>',
 '<start> huawei nova lol <end>',
 '<start> monday <end>',
 '<start> on monday you will hear of me . <end>',
 '<start> i am a fellow . r praisethecube and i agree to alliance against r shrineofdurr i am a cube ambassador seeking peace with the tomato heads <end>',
 '<start> here s a sneak peek of r praisethecube using the of all time ! m , <end>',
 '<start> h <end>',
 '<start> i <end>',
 '<start> i timed this to be right before the switch so it should work <end>',
 '<start> yeet i won <end>',
 '<start> not yet . <end>',
 '<start> it s treason then <end>',
 '<start> is it possible to learn this power <end>',
 '<start> not from a jedi <end>',
 '<start> like drugs or what yes dm him if u want <end>',
 '<start> yeah i ll dm him soon <end>',
 '<start> l <end>',
 '<start> i <end>',
 '<start> should have kept your fingers crossed . i think it s cooties . <end>',
 '<start> hes doomed , no medical professional could help now <end>',
 '<start> yeah , uk

In [54]:
re.sub(r'(\[[a-zA-Z0-9 ]*\]\(https:\/\/[\/?=&;a-zA-Z0-9.]*\))', " \1 ", x)

"Here's a sneak peek of /r/PraiseTheCube using the  \x01  of all time! newlinechar  newlinechar \\#1:  \x01  | [8 comments](https://np.reddit.com/r/PraiseTheCube/comments/9bdz7s/praise_the_c_u_b_e/)   newlinechar \\#2: [praise the ć̇̌̑̏ͫ͋ͤ̆ͮͣ͂̈́̈́͐̆̐͑͏̴̞̝͖̞͢ų̴͉͖̩̙̝̘̖̥̤̘̂͑̌ͦ́͘͝ͅͅb̢̛ͥ̅ͨ̿̅ͩͩ̈́̒ͯ̀ͬ̿̒͂̿̈̀͘͏̥̰̣̬̲̩̺̹̱̮̙͖̬ë̸̇͑̓ͫͬ̌̂͏̵̡͕͙̭̤̦̳̲̠͔͟ͅ, for he has given me trickshot powers](https://gfycat.com/WanRashFlyingsquirrel) | [22 comments](https://np.reddit.com/r/PraiseTheCube/comments/9b973i/praise_the_cube_for_he_has_given_me_trickshot/)   newlinechar \\#3: [The Cube is love, The Cube is life.](https://i.redd.it/vfep3b9zeri11.jpg) | [5 comments](https://np.reddit.com/r/PraiseTheCube/comments/9avtgb/the_cube_is_love_the_cube_is_life/) newlinechar  newlinechar ---- newlinechar ^^I'm ^^a ^^bot, ^^beep ^^boop ^^| ^^Downvote ^^to ^^remove ^^| [^^Contact ^^me](https://www.reddit.com/message/compose/?to=sneakpeekbot) ^^| [^^Info](https://np.reddit.com/r/sneakpeekbot/) ^^| [^^Opt-out](https://np.re

In [8]:
import math
corpus_size = 5027205
epoch  = 0
hparams = {"batch_size": 32}
num_train_steps = math.ceil((epoch + 1) * corpus_size / (hparams['batch_size'] if 'batch_size' in hparams else 128))
num_train_steps

157101

In [5]:
5027205/32


157100.15625

In [10]:
157100*3

471300